In [50]:

import numpy as np
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
import string
from gensim.models import Word2Vec
import joblib
import re
import gradio as gr

# Load your trained Word2Vec model
w2v_model = Word2Vec.load('/work/NLP_Project/GenreFromLyricsShared/Word2VecModels/original_w2v.model')

def preprocess(lyrics):
    #stop = stopwords.words('english')
    lyrics = re.sub('\n', ' ', lyrics)
    lyrics = re.sub('\[.*?\]', '', lyrics)
    lyrics = re.sub('\\[^\s]*', '', lyrics)
    lyrics = re.sub("'", '', lyrics)
    lyrics = lyrics.strip()
    lyrics = re.sub(r'\s+', ' ', lyrics)
    lyrics = re.sub('['+string.punctuation+']', '', lyrics)
    lyrics = lyrics.lower()
    lyrics = re.sub('chorus', '', lyrics)
    lyrics = re.sub('verse', '', lyrics)
    #lyrics = ' '.join([word for word in lyrics.split() if word not in stop])
    tokens = nltk.word_tokenize(lyrics)

    # Use Word2Vec model to generate word vectors and then calculate the mean vector
    vector = np.mean([w2v_model.wv[word] for word in tokens if word in w2v_model.wv.key_to_index], axis=0)
    return vector

def predict_genre(Artist, Title, Lyrics, File):
    # Preprocess the lyrics
    vector = preprocess(lyrics)

    # Reshape the vector to 2D array as the model.predict expects 2D array
    vector = vector.reshape(1, -1)

    # Load the trained Random Forest Classifier
    clf = joblib.load('/work/NLP_Project/GenreFromLyricsShared/random_forest.sav')

    # Get the probabilities of each class
    probabilities = clf.predict_proba(vector)[0]

    # Map each probability with the corresponding genre
    genres = clf.classes_
    result = {genre: prob for genre, prob in zip(genres, probabilities)}

    return result

description = '<img src="https://storage.googleapis.com/pr-newsroom-wp/1/2018/11/Spotify_Logo_CMYK_Green.png" alt="Spotify Logo">'

iface = gr.Interface(
    fn=predict_genre,
    inputs=[
        gr.inputs.Textbox(lines=1, placeholder='Artist Here...'),
        gr.inputs.Textbox(lines=1, placeholder='Title Here...'),
        gr.inputs.Textbox(lines=4, placeholder='Lyrics Here...'),
        #gr.inputs.File()
    ],
    outputs=gr.outputs.Label(label="Genre Suggestion"),
    description=description
)
iface.launch(debug=True, share=True)



[nltk_data] Downloading package punkt to /home/ucloud/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/ucloud/.local/lib/python3.10/site-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/home/ucloud/.local/lib/python3.10/site-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
/home/ucloud/.local/lib/python3.10/site-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/home/ucloud/.local/lib/python3.10/site-packages/gradio/outputs.py:200: UserWarning: The 'type' p


Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Running on local URL:  http://127.0.0.1:7864


KeyboardInterrupt: 

In [52]:
predict_genre("X", "X", "hello", "X")

[Parallel(n_jobs=31)]: Using backend ThreadingBackend with 31 concurrent workers.
[Parallel(n_jobs=31)]: Done  90 out of 100 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=31)]: Done 100 out of 100 | elapsed:    0.0s finished


{'country': 0.05, 'pop': 0.47, 'rap': 0.19, 'rb': 0.07, 'rock': 0.22}

In [67]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import gradio as gr
from transformers import BertTokenizer
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
import torchmetrics

# Load BERT model and tokenizer
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class LyricsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {'input_ids': torch.as_tensor(self.encodings.iloc[idx])}
        item['labels'] = torch.as_tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.encodings)

class LyricsClassifier(pl.LightningModule):
    def __init__(self, model_name='bert-base-uncased', num_labels=5): #@RIES TRY "bert-large-uncased" with the A100
        super().__init__()
        self.save_hyperparameters()
        self.bert = BertForSequenceClassification.from_pretrained(self.hparams.model_name,
                                                                  num_labels=self.hparams.num_labels)
        self.accuracy = torchmetrics.Accuracy(task="multiclass",compute_on_step=False, num_classes=num_labels)

    def forward(self, input_ids, labels=None):
        return self.bert(input_ids, labels=labels)
    
    def training_step(self, batch, batch_idx):
        outputs = self.forward(batch['input_ids'], batch['labels'])
        loss = outputs.loss
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self.forward(batch['input_ids'], batch['labels'])
        _, predicted = torch.max(outputs.logits, 1)
        correct = (predicted == batch['labels']).sum().item()
        accuracy = correct / len(batch['labels'])
        self.log('val_accuracy', accuracy, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return accuracy
        
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=1e-5)

model = LyricsClassifier.load_from_checkpoint(checkpoint_path="/work/NLP_Project/GenreFromLyricsShared/epoch=1-step=145040.ckpt", map_location=torch.device('cpu'))

def strip_lyrics(lyrics):
    # Remove strings enclosed in brackets []
    lyrics = re.sub(r'\[.*?\]', '', lyrics)
    
    # Remove substrings starting with a backslash \
    lyrics = re.sub(r'\\[^\s]*', '', lyrics)

    # Remove newline characters \n
    lyrics = re.sub(r'\n', ' ', lyrics)
    
    # Remove single quotes '
    lyrics = re.sub(r"'", '', lyrics)
    
    # Remove leading and trailing whitespaces
    lyrics = lyrics.strip()

    # Strip the string and ensure only one space between words
    lyrics = re.sub(r'\s+', ' ', lyrics.strip())

    return lyrics

def predict_genre(Artist, Title, Lyrics):
    lyrics = strip_lyrics(lyrics)  # Preprocess the lyrics
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    inputs = tokenizer(lyrics, return_tensors="pt", padding=True, truncation=True, max_length=512)
    input_ids = inputs['input_ids'].to(model.device)
    
    with torch.no_grad():
        outputs = model(input_ids)
    
    probabilities = F.softmax(outputs.logits, dim=-1)
    probabilities = probabilities.cpu().numpy()

    label_map = {0: 'Country', 1: 'Pop', 2: 'Rap', 3: 'R&B', 4: 'Rock'}
    
    probabilities_dict = {label_map[i]: float(prob) for i, prob in enumerate(probabilities[0])}  # convert numpy.float32 to float

    return probabilities_dict




description = '<img src="https://storage.googleapis.com/pr-newsroom-wp/1/2018/11/Spotify_Logo_CMYK_Green.png" alt="Spotify Logo">'

iface = gr.Interface(
    fn=predict_genre,
    inputs=[
        gr.inputs.Textbox(lines=1, placeholder='Artist Here...'),
        gr.inputs.Textbox(lines=1, placeholder='Title Here...'),
        gr.inputs.Textbox(lines=4, placeholder='Lyrics Here...'),
        #gr.inputs.File()
    ],
    outputs=gr.outputs.Label(num_top_classes=5, label="Genre Suggestion"),
    description=description
)
iface.launch(debug=True, share=True)





Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Running on local URL:  http://127.0.0.1:7865
Running on public URL: https://62b589753eec286911.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7865 <> https://62b589753eec286911.gradio.live


In [61]:
# Example usage
text = "Hank, Beugard, Big Sur Y'all know how this - go, you know All eyes on me, OG Roll up in the club and shit (is that right?) All eyes on me All eyes on me Ay you know what? I bet you got it twisted, you don't know who to trust So many player-hatin' - tryna sound like us Say they ready for the funk, but I don't think they knowin' Straight to the depths of Hell is where them cowards goin' Well, are you still down? Holla when you see me And let these devils be sorry for the day they finally freed me I got a caravan of - every time we ride Hittin' - up when we pass by Until I die, live the life of a boss player, 'cause even when I'm high F- with me and get crossed later, the futures in my eyes 'Cause all I want is cash and thangs A five-double-oh Benz, flauntin' flashy rings Uhh, - pursue me like a dream Been known to disappear before your eyes just like a dope fiend It seems, my main thing was to be major paid The game sharper than a - razor blade Say money bring - bring lies One - gettin' jealous and - die Depend on me like the first and fifteenth They might hold me for a second, but these punks won't get me We got four niggas in low riders and ski masks Screamin', Thug Life every time they pass, all eyes on me Live the life of a thug - until the day I die Live the life of a boss player All eyes on me All eyes on me Live the life of a thug - until the day I die Live the life of a boss player 'cause even gettin' high"

#strip lyrics
text_stripped = strip_lyrics("hi", "hi", text)

probabilities = predict_genre(text_stripped)

TypeError: strip_lyrics() takes 1 positional argument but 3 were given

In [49]:
probabilities

{'Country': 0.0007394675,
 'Pop': 0.24420594,
 'Rap': 0.737701,
 'R&B': 0.0054926835,
 'Rock': 0.011860887}